# CS 5228


In [1]:
# All Imports
from utils import *
import pandas as pd
import locale
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing

# model training
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# model evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.tree import DecisionTreeClassifier # decision Tree
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import *

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_ALL,'')
pd.set_option('display.max_columns', None)

In [2]:
# Process Training Data
# drop_columns = ['CreateJob','RetainedJob','City','Name','Zip','BankState']

# drop_columns = ['CreateJob','ApprovalFY','ApprovalDate','DisbursementGross','SBA_Appv']

drop_columns = []

le = generate_labels()
sc = generate_scaler(le, preprocessing.MinMaxScaler())

base_dropna = get_data(scaler=sc,le=le,type='train', dropna=True, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)

In [3]:
base_dropna.head(20)

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,FranchiseCode,DisbursementDate,DisbursementGross,GrAppv,SBA_Appv,NewExist_1,NewExist_2,UrbanRural_0,UrbanRural_1,UrbanRural_2,RevLineCr_N,RevLineCr_Y,LowDoc_N,LowDoc_Y,ChargeOff
0,0.645099,0.714894,0.18,0.330273,0.858709,0.156863,0.360423,0.911156,0.911111,0.321782,0.002500,0.000227,0.000000,0.000000,0.911940,0.400000,0.399976,0.333319,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0
1,0.498425,0.778173,0.88,0.840948,1.000000,0.882353,0.254401,0.906528,0.911111,0.081683,0.000600,0.000114,0.000341,0.000000,0.906513,0.003400,0.003360,0.003756,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1
2,0.288509,0.489379,0.08,0.900229,0.077249,0.529412,0.874273,0.739557,0.733333,0.148515,0.000100,0.000114,0.000114,0.000000,0.741188,0.003400,0.003360,0.001867,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0
3,0.766940,0.485436,0.96,0.535555,0.064291,0.960784,0.000000,0.558641,0.555556,0.396040,0.000100,0.000000,0.000000,0.000000,0.561105,0.027400,0.027361,0.027378,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0
4,0.028081,0.470936,0.48,0.655367,0.232993,0.470588,0.669321,0.817988,0.822222,0.250000,0.000200,0.000795,0.000227,0.000000,0.820954,0.009333,0.004960,0.002756,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0
5,0.200377,0.667260,0.10,0.811478,0.431597,0.098039,0.479798,0.861223,0.866667,0.673267,0.000300,0.000114,0.000227,0.000000,0.869252,0.080000,0.099964,0.083313,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1
6,0.637809,0.466675,0.66,0.891169,0.974084,0.823529,0.874143,0.741932,0.733333,0.742574,0.000500,0.000568,0.000568,0.000000,0.744908,0.055000,0.054962,0.045812,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0
7,0.780104,0.256042,0.40,0.210432,0.527037,0.392157,0.456773,0.905614,0.911111,0.262376,0.000100,0.000000,0.000114,0.000000,0.906513,0.015000,0.014961,0.014978,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1
8,0.587171,0.161346,0.08,0.945189,0.077249,0.666667,0.669321,0.680490,0.688889,0.742574,0.000500,0.000000,0.000568,0.000000,0.681790,0.139860,0.139826,0.116530,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0
9,0.422661,0.028937,0.70,0.453045,0.154996,0.078431,0.521615,0.839118,0.844444,0.024752,0.001500,0.000568,0.001705,0.000000,0.839615,0.020000,0.019961,0.011089,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1


In [4]:
base_dropna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49808 entries, 0 to 49807
Data columns (total 28 columns):
Name                 49808 non-null float64
City                 49808 non-null float64
State                49808 non-null float64
Zip                  49808 non-null float64
Bank                 49808 non-null float64
BankState            49808 non-null float64
NAICS                49808 non-null float64
ApprovalDate         49808 non-null float64
ApprovalFY           49808 non-null float64
Term                 49808 non-null float64
NoEmp                49808 non-null float64
CreateJob            49808 non-null float64
RetainedJob          49808 non-null float64
FranchiseCode        49808 non-null float64
DisbursementDate     49808 non-null float64
DisbursementGross    49808 non-null float64
GrAppv               49808 non-null float64
SBA_Appv             49808 non-null float64
NewExist_1           49808 non-null float64
NewExist_2           49808 non-null float64
UrbanRural_

In [5]:
# Process Test Data
base_test = get_data(scaler=sc,le=le,type='test', dropna=False, get_dummy=True, feature_split=False, values_only=True,drop_columns=drop_columns)

In [6]:
base_dropna.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49808 entries, 0 to 49807
Data columns (total 28 columns):
Name                 49808 non-null float64
City                 49808 non-null float64
State                49808 non-null float64
Zip                  49808 non-null float64
Bank                 49808 non-null float64
BankState            49808 non-null float64
NAICS                49808 non-null float64
ApprovalDate         49808 non-null float64
ApprovalFY           49808 non-null float64
Term                 49808 non-null float64
NoEmp                49808 non-null float64
CreateJob            49808 non-null float64
RetainedJob          49808 non-null float64
FranchiseCode        49808 non-null float64
DisbursementDate     49808 non-null float64
DisbursementGross    49808 non-null float64
GrAppv               49808 non-null float64
SBA_Appv             49808 non-null float64
NewExist_1           49808 non-null float64
NewExist_2           49808 non-null float64
UrbanRural_

In [ ]:
base_dropna_x = base_dropna.drop(columns='ChargeOff')
base_dropna_y = base_dropna['ChargeOff']

In [21]:
# Train Model
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=1234,),random_state=1234)

param_grid = {'learning_rate': [0.05,0.1,0.2],
#               'loss': ['exponential'],
#               'max_depth':[8],
#               'max_features':[None],
              'n_estimators':[300,350,400],
#               'min_samples_split':[2],
#               "base_estimator__min_samples_split":[2,3,4],
#               "base_estimator__criterion":["gini", "entropy"],
              "base_estimator__max_depth":[2,3,4,5]
             }

model = GridSearchCV(clf, param_grid,scoring = 'accuracy', cv=5,n_jobs=-1,verbose=10)
model.fit(base_dropna_x, base_dropna_y)


print("Best Accuracy :",model.best_score_)
print("Best Parameters",model.best_params_)
print(model.cv_results_)

# Prediction
test_pred = model.predict(base_test)
pd.DataFrame(test_pred).to_csv('/ada/y_pred1.csv',header=['ChargeOff'],index_label="Id")

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   29.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  89 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 121 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 176 out of 180 | elapsed: 13.9min remaining:   19.0s
[Parallel(n_jobs=-1)]: Done 180 out 

Best Accuracy : 0.9229041102097014
Best Parameters {'base_estimator__max_depth': 3, 'learning_rate': 0.2, 'n_estimators': 350}


FileNotFoundError: [Errno 2] No such file or directory: 'ada/y_pred1.csv'

In [24]:
# Train Model
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=1234,),random_state=1234)

param_grid = {'learning_rate': [0.2],
#               'loss': ['exponential'],
#               'max_depth':[8],
#               'max_features':[None],
              'n_estimators':[350],
#               'min_samples_split':[2],
              "base_estimator__min_samples_split":[2,3,4,5],
              "base_estimator__criterion":["gini", "entropy"],
              "base_estimator__max_depth":[3]
             }

model = GridSearchCV(clf, param_grid,scoring = 'accuracy', cv=5,n_jobs=-1,verbose=10)
model.fit(base_dropna_x, base_dropna_y)


print("Best Accuracy :",model.best_score_)
print("Best Parameters",model.best_params_)
print(model.cv_results_)

# Prediction
test_pred = model.predict(base_test)
pd.DataFrame(test_pred).to_csv('ada/y_pred2.csv',header=['ChargeOff'],index_label="Id")

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   49.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   49.8s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  22 out of  40 | elapsed:  1.8min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  27 out of  40 | elapsed:  2.6min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  32 out of  40 | elapsed:  2.6min remaining:   38.8s
[Parallel(n_jobs=-1)]: Done  37 out of  40 | elapsed:  3.2min remaining:   15.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  3.2min finished


Best Accuracy : 0.9229041102097014
Best Parameters {'base_estimator__criterion': 'gini', 'base_estimator__max_depth': 3, 'base_estimator__min_samples_split': 2, 'learning_rate': 0.2, 'n_estimators': 350}
{'mean_fit_time': array([48.73901601, 48.6772964 , 49.05315022, 49.22199903, 54.76979733,
       54.3924705 , 53.97034321, 39.28947659]), 'std_fit_time': array([0.08288331, 0.04692478, 0.30854431, 0.07593214, 0.17935819,
       0.11633856, 0.39997469, 7.14019661]), 'mean_score_time': array([0.96256018, 0.96191158, 0.93513899, 0.94018312, 0.69870596,
       0.74702883, 0.61286201, 0.35267882]), 'std_score_time': array([0.0321902 , 0.01395241, 0.06494668, 0.06377983, 0.01556035,
       0.01113027, 0.11268635, 0.07315612]), 'param_base_estimator__criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'entropy', 'entropy',
                   'entropy', 'entropy'],
             mask=[False, False, False, False, False, False, False, False],
       fill_value='?',
            dtype=ob

In [28]:
# Train Model
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=1234,),random_state=1234)

param_grid = {'learning_rate': [0.2],
#               'loss': ['exponential'],
#               'max_features':[None],
              'n_estimators':range(350,450,10),
              "base_estimator__min_samples_split":[2],
              "base_estimator__criterion":["gini"],
              "base_estimator__max_depth":[3]
             }

model = GridSearchCV(clf, param_grid,scoring = 'accuracy', cv=5,n_jobs=-1,verbose=10)
model.fit(base_dropna_x, base_dropna_y)


print("Best Accuracy :",model.best_score_)
print("Best Parameters",model.best_params_)
print(model.cv_results_)

# Prediction
test_pred = model.predict(base_test)
pd.DataFrame(test_pred).to_csv('ada/y_pred3.csv',header=['ChargeOff'],index_label="Id")

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   51.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  33 out of  50 | elapsed:  2.7min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  39 out of  50 | elapsed:  3.6min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed:  3.7min remaining:   24.5s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.3min finished


Best Accuracy : 0.9238276497775171
Best Parameters {'base_estimator__criterion': 'gini', 'base_estimator__max_depth': 3, 'base_estimator__min_samples_split': 2, 'learning_rate': 0.2, 'n_estimators': 430}
{'mean_fit_time': array([48.3851428 , 49.87296762, 50.97232304, 52.22530918, 53.67036963,
       54.58244596, 55.979599  , 56.95616326, 57.90972157, 50.95399981]), 'std_fit_time': array([0.09719947, 0.14370936, 0.20535975, 0.06929279, 0.19639597,
       0.04728239, 0.07022947, 0.15236024, 0.18522668, 8.586226  ]), 'mean_score_time': array([0.68956718, 0.69375119, 0.6669569 , 0.66843638, 0.67401237,
       0.69061904, 0.69819875, 0.69891996, 0.53761902, 0.36943202]), 'std_score_time': array([0.00331091, 0.00921318, 0.00794359, 0.01152307, 0.01698454,
       0.01061942, 0.00937384, 0.03451721, 0.07703398, 0.00412788]), 'param_base_estimator__criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'gini', 'gini', 'gini'],
             mask

In [31]:
# Train Model
clf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=1234,),random_state=1234)

param_grid = {'learning_rate': [0.2,0.25,0.3,0.35,0.4],
#               'loss': ['exponential'],
#               'max_features':[None],
              'n_estimators':[430],
              "base_estimator__min_samples_split":[2],
              "base_estimator__criterion":["gini"],
              "base_estimator__max_depth":[3],
              "base_estimator__splitter":['best','random']
             }

print(param_grid)

model = GridSearchCV(clf, param_grid,scoring = 'accuracy', cv=10,n_jobs=-1,verbose=10)
model.fit(base_dropna_x, base_dropna_y)


print("Best Accuracy :",model.best_score_)
print("Best Parameters",model.best_params_)
print(model.cv_results_)

# Prediction
test_pred = model.predict(base_test)
pd.DataFrame(test_pred).to_csv('ada/y_pred4.csv',header=['ChargeOff'],index_label="Id")

{'learning_rate': [0.2, 0.25, 0.3, 0.35, 0.4], 'n_estimators': [430], 'base_estimator__min_samples_split': [2], 'base_estimator__criterion': ['gini'], 'base_estimator__max_depth': [3], 'base_estimator__splitter': ['best', 'random']}
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done  88 out of 100 | elapsed:  6.4min remaining:   52.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  6.7min finished


Best Accuracy : 0.9255543192646112
Best Parameters {'base_estimator__criterion': 'gini', 'base_estimator__max_depth': 3, 'base_estimator__min_samples_split': 2, 'base_estimator__splitter': 'best', 'learning_rate': 0.25, 'n_estimators': 430}
{'mean_fit_time': array([66.44006331, 66.33148344, 66.05622444, 65.87732062, 69.81422126,
       26.91762226, 27.6398453 , 28.17872977, 31.39885409, 21.4949955 ]), 'std_fit_time': array([0.10298994, 0.12701434, 0.17489893, 0.09539147, 7.6612621 ,
       0.05879164, 0.10692899, 0.10996554, 0.07157007, 4.94140104]), 'mean_score_time': array([0.52110112, 0.49659801, 0.49061091, 0.4757802 , 0.53228209,
       0.54060848, 0.57105699, 0.57928503, 0.62231996, 0.36444919]), 'std_score_time': array([0.0183049 , 0.02745471, 0.03048209, 0.03847771, 0.04740488,
       0.00796483, 0.00712194, 0.0159863 , 0.00701228, 0.13191087]), 'param_base_estimator__criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'gini

In [ ]:
results = model.cv_results_
plt.figure(figsize=(13, 13))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
          fontsize=16)

plt.xlabel("n_estimators")
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(0, 402)
ax.set_ylim(0.73, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results['n_estimators'].data, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k']):
    for sample, style in (('train', '--'), ('test', '-')):
        sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
        sample_score_std = results['std_%s_%s' % (sample, scorer)]
        ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                        sample_score_mean + sample_score_std,
                        alpha=0.1 if sample == 'test' else 0, color=color)
        ax.plot(X_axis, sample_score_mean, style, color=color,
                alpha=1 if sample == 'test' else 0.7,
                label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
            linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score,
                (X_axis[best_index], best_score + 0.005))

plt.legend(loc="best")
plt.grid(False)
plt.show()

In [ ]:
full_x.shape

In [23]:
pd.DataFrame(test_pred).to_csv('ada/y_pred1.csv',header=['ChargeOff'],index_label="Id")